In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import pickle as pk

In [2]:
train_df = pd.read_csv('data/fraudTrain.csv')
test_df = pd.read_csv('data/fraudTest.csv')
train_df.drop(columns=['Unnamed: 0'],inplace=True)
test_df.drop(columns=['Unnamed: 0'],inplace=True)

In [3]:
def post_process(data):
    if data <= 0.5:
        data = "Not Fraud"
    else:
        data = "Fraud"
    return data

In [4]:
def pre_year(data):    
    data = data.split("-")
    return data [0]

In [5]:
def pre_hour(data):
    data = data.split(":")
    return data [0]

In [6]:
def pre_age(data):
    age = 2021 - data
    return age

In [7]:
def pre_time(data):
    data = data.split(" ")
    return data[1]

In [8]:
def transform(df):

    df['amt'] = df['amt'].astype(int)
    df['city_pop'] = df['city_pop'].astype(int)
    df['year'] = df['dob'].apply(pre_year).astype(int)
    df['age'] = df['year'].apply(pre_age).astype(int)
    df['time_old'] = df['trans_date_trans_time'].apply(pre_time)
    df['time'] = df['time_old'].apply(pre_hour).astype(int)
    df['is_fraud'] = df['is_fraud'].astype(int)
    
    return df

In [9]:
train_df = transform(train_df)
test_df = transform(test_df)

In [10]:
feature_columns = ['time', 'amt', 'city_pop', 'age']
label_column = ['is_fraud']

train_X = train_df[feature_columns].to_numpy()
train_y = train_df[label_column].to_numpy().ravel()

test_X = test_df[feature_columns].to_numpy()
test_y = test_df[label_column].to_numpy().ravel()

In [11]:
%%time
rf = RandomForestClassifier(n_estimators=10, max_depth=3, random_state=6)
model = rf.fit(train_X,train_y)

CPU times: user 6.45 s, sys: 120 ms, total: 6.57 s
Wall time: 6.57 s


In [12]:
%%time
rf.score(test_X,test_y)

CPU times: user 220 ms, sys: 16.4 ms, total: 236 ms
Wall time: 234 ms


0.9970524671641603

In [13]:
with open('rf_model.pkl','wb') as f:
    pk.dump(model,f)